In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
from tqdm import tqdm
import torch.nn.functional as F

### Utility function

In [2]:
def preprocessing(arguments, key_points, labels):
    columns = ['topic', 'premise', 'hypothesis', 'label']
    data = []
    for index, row in labels.iterrows():
        argument_row = arguments.loc[arguments['arg_id'] == row['arg_id']]
        topic = argument_row['topic'].iloc[0]
        premise = argument_row['argument'].iloc[0]
        kp_row = key_points.loc[key_points['key_point_id'] == row['key_point_id']]
        hypothesis = kp_row['key_point'].iloc[0]
        label = row['label']
        data.append([topic, premise, hypothesis, label])

    result = pd.DataFrame(data, columns=columns)
    return result

### Create Torch Dataset

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, text_df, tokenizer, max_length):
        self.premises = list(text_df['premise'].values+tokenizer.sep_token+text_df['topic'].values)
        self.hypotheses = list(text_df['hypothesis'].values+tokenizer.sep_token+text_df['topic'].values)
        self.labels = list(text_df['label'])
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.premises)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.premises[idx],
            self.hypotheses[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        return item


### Define the LightningModule

In [4]:
class KPM(pl.LightningModule):
    def __init__(self, model, learning_rate=2e-5, weight_decay=0.001):
        super(KPM, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        outputs = self(input_ids, attention_mask, labels)
        # print("--------------training-------------------------------")
        # print("Shape: "+ str(outputs.logits.shape))
        # print(outputs.logits)
        self.log("train_loss", torch.clone(outputs.loss).detach())
        one_hot_labels = torch.stack([1 - labels, labels], dim=-1)
        one_hot_labels = one_hot_labels.float()
        # loss = outputs.loss
        loss = F.binary_cross_entropy_with_logits(outputs.logits, one_hot_labels, reduction='mean')
        self.log("train_loss", torch.clone(loss).detach())
        return loss

    # def on_validation_batch_start(self, batch, batch_idx, dataloader_idx):
    #     input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
    #     print(len(input_ids))
    #     # inputs = batch[:-1]  # Assuming the input tensor is at the first index
    #     # print(f"Validation Batch {batch_idx + 1}, Input Tensor Size: {inputs[0].size()}")
    #     # print(inputs[0])  # Assuming the input tensor is at index 0

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", torch.clone(outputs.loss).detach())
        one_hot_labels = torch.stack([1 - labels, labels], dim=-1)
        one_hot_labels = one_hot_labels.float()
        # print(labels)
        # loss = outputs.loss
        loss = F.binary_cross_entropy_with_logits(outputs.logits, one_hot_labels, reduction='mean')
        self.log("val_loss", torch.clone(loss).detach())
        return  loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

### Import Dataset and Preprocessing

In [5]:
ArgKP2021_topics = [
    "Assisted suicide should be a criminal offence",
    "Homeschooling should be banned",
    "The vow of celibacy should be abandoned",
    "We should abandon marriage",
    "We should abandon the use of school uniform",
    "We should abolish capital punishment",
    "We should abolish intellectual property rights",
    "We should abolish the right to keep and bear arms",
    "We should adopt an austerity regime",
    "We should adopt atheism",
    "We should adopt libertarianism",
    "We should ban human cloning",
    "We should ban private military companies",
    "We should ban the use of child actors",
    "We should close Guantanamo Bay detention camp",
    "We should end affirmative action",
    "We should end mandatory retirement",
    "We should fight for the abolition of nuclear weapons",
    "We should fight urbanization",
    "We should introduce compulsory voting",
    "We should legalize cannabis",
    "We should legalize prostitution",
    "We should legalize sex selection",
    "We should prohibit flag burning",
    "We should prohibit women in combat",
    "We should subsidize journalism",
    "We should subsidize space exploration",
    "We should subsidize vocational education",
    "Routine child vaccinations should be mandatory",
    "Social media platforms should be regulated by the government",
    "The USA is a good country to live in"
]

In [6]:
dev_topics = [
    ArgKP2021_topics[24],
    ArgKP2021_topics[25],
    ArgKP2021_topics[26],
    ArgKP2021_topics[27]
]
print(dev_topics)

['We should prohibit women in combat', 'We should subsidize journalism', 'We should subsidize space exploration', 'We should subsidize vocational education']


In [7]:

# labels_df = pd.read_csv('./data/labels_train+dev.csv')
# print(labels_df.columns)
# argument_df = pd.read_csv('./data/arguments_train+dev.csv')
# for index, row in labels_df.iterrows():
#         argument_row = argument_df.loc[argument_df['arg_id'] == row['arg_id']]
#         topic = argument_row['topic'].iloc[0]
#         labels_df.at[index, 'topic'] = topic
# labels_df.to_csv('./data/labels_train+dev.csv',index=False)
# print("finished")
# labels_df= labels_df.drop(labels_df.columns[0], axis=1)
# print(labels_df.columns)
# labels_df.to_csv('./data/labels_train+dev.csv',index=False)
# print("finished")

In [8]:
argument_df = pd.read_csv('./data/arguments_train+dev.csv')
kp_df = pd.read_csv('./data/key_points_train+dev.csv')
labels_df = pd.read_csv('./data/labels_train+dev.csv')


val_argument_df = argument_df[argument_df['topic'].isin(dev_topics)]
val_kp_df = kp_df[kp_df['topic'].isin(dev_topics)]
val_label_df = labels_df[labels_df['topic'].isin(dev_topics)]
val_df = preprocessing(val_argument_df, val_kp_df, val_label_df)
print(val_argument_df)


train_argument_df = argument_df[~argument_df.isin(val_argument_df.to_dict(orient='list')).all(1)]
train_kp_df = kp_df[~kp_df.isin(val_kp_df.to_dict(orient='list')).all(1)]
train_label_df = labels_df[~labels_df.isin(val_label_df.to_dict(orient='list')).all(1)]
train_df = preprocessing(train_argument_df, train_kp_df, train_label_df)
print(train_argument_df)

test_argument_df = pd.read_csv('./data/arguments_test.csv')
test_kp_df = pd.read_csv('./data/key_points_test.csv')
test_label_df = pd.read_csv('./data/labels_test.csv')
test_df = preprocessing(test_argument_df, test_kp_df, test_label_df)

          arg_id                                           argument  \
5580    arg_24_0  all citizens should be given equal opportunity...   
5581    arg_24_1  allowing women in combat gives us the numbers ...   
5582    arg_24_2  anyone who is physically qualified should be a...   
5583    arg_24_3  as long as women are fully trained there are m...   
5584    arg_24_4  combat is never an individual against another ...   
...          ...                                                ...   
6510  arg_27_218  we should subsidize vocational education to en...   
6511  arg_27_219  We should subsidize vocational education to su...   
6512  arg_27_220  While many who graduate from universities stru...   
6513  arg_27_221  with the rising cost of college tuition vocati...   
6514  arg_27_222  yes, we should subsidize vocational education ...   

                                         topic  stance  
5580        We should prohibit women in combat      -1  
5581        We should prohibit wo

### Training phase

In [9]:
model_name = "cross-encoder/nli-distilroberta-base"
num_classes = 2
max_length = 512
batch_size = 16
learning_rate = 5e-05
weight_decay = 0.001

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_classes, ignore_mismatched_sizes = True)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
# Change the loss function to binary cross-entropy

# Get the vocabulary
vocab = tokenizer.get_vocab()

# Check if the "CLS" token is present in the vocabulary
cls_token_present = tokenizer.cls_token in vocab
sep_token_present = tokenizer.sep_token in vocab
print("Is CLS token present in the vocabulary?", cls_token_present)
print("Is SEP token present in the vocabulary?", sep_token_present)

train_dataset = Dataset(train_df, tokenizer, max_length)
val_dataset = Dataset(val_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = KPM(model, learning_rate)

# Define a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoint',
    filename='nli_model-{epoch:02d}-{val_loss:.2f}-24_25_26_27',
    save_top_k=1,
    mode='min'
)

early_stopping =EarlyStopping(
    monitor="val_loss",
    min_delta=0.01,
    patience=3
)

# define trainer
trainer = Trainer(
    min_epochs = 0, # change this
    max_epochs = 20,
    callbacks=[checkpoint_callback, early_stopping],
    accelerator="auto",
    #progress_bar_refresh_rate=30,
    #gpus = 1 if device.type == 'cuda' else 0
    devices = 1 if torch.cuda.is_available() else None
)

torch.cuda.empty_cache()

# start training
trainer.fit(model, train_loader, val_loader)
print("Training is finished")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Is CLS token present in the vocabulary? True
Is SEP token present in the vocabulary? True


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name  | Type                             | Params
-----------------------------------------------------------
0 | model | RobertaForSequenceClassification | 82.1 M
-----------------------------------------------------------
82.1 M    Trainable params
0         Non-trainable params
82.1 M    Total params
328.480   Total estimated model params size (MB)
D:\anaconda3\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:631: UserWarning: Checkpoint directory ./checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint dire

Validation sanity check: 0it [00:00, ?it/s]

D:\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Training is finished


### Prediction and Evaluation

In [ ]:
# Assuming you have already trained the model and saved a checkpoint
checkpoint_path = './checkpoint/nli_model-epoch=01-val_loss=0.37.ckpt'
# Load the model from the checkpoint
model_name = "cross-encoder/nli-distilroberta-base"
num_classes = 2
max_length = 512
batch_size = 16
learning_rate = 5e-05
weight_decay = 0.001

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_classes, ignore_mismatched_sizes = True)
loaded_model = KPM.load_from_checkpoint(model=model, checkpoint_path=checkpoint_path)
print("hello world")

# Set the model to evaluation mode
loaded_model.eval()

test_dataset = Dataset(test_df, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Make predictions on the test set with probabilities
all_predictions = []
all_probabilities = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        input_ids, attention_mask = batch['input_ids'], batch['attention_mask']
        outputs = loaded_model(input_ids, attention_mask)
        logits = outputs.logits
        # print(logits)
        probabilities = F.softmax(logits, dim=1)
        predictions = torch.argmax(logits, dim=1).tolist()
        # print(predictions)
        print(probabilities)
        all_predictions.extend(predictions)
        all_probabilities.extend(probabilities.tolist())


print(all_predictions)
print(all_probabilities)

In [ ]:
print(len(all_predictions))
print(len(all_probabilities))

In [14]:
print(all_probabilities)
all_probabilities = [max(sublist) for sublist in all_probabilities]
print(all_probabilities)

[0.8513832688331604, 0.9530447125434875, 0.9813437461853027, 0.9999032020568848, 0.9502105712890625, 0.9856218695640564, 0.9996399879455566, 0.7003836631774902, 0.9784537553787231, 0.9999059438705444, 0.5496004223823547, 0.9834389090538025, 0.9875774383544922, 0.9435656070709229, 0.9999083280563354, 0.8970844745635986, 0.9932764768600464, 0.5377925038337708, 0.9752899408340454, 0.9779468178749084, 0.9998916387557983, 0.6552639603614807, 0.9617480635643005, 0.9989585876464844, 0.9997628331184387, 0.9690905809402466, 0.7799464464187622, 0.9968823194503784, 0.9996205568313599, 0.7533658742904663, 0.9707180857658386, 0.9991835951805115, 0.9999151229858398, 0.6591960191726685, 0.9929141998291016, 0.6223980188369751, 0.5727285146713257, 0.9057391285896301, 0.9987348914146423, 0.6909785270690918, 0.9980675578117371, 0.9997209906578064, 0.99972003698349, 0.9550588130950928, 0.7782137989997864, 0.5976061224937439, 0.9899438619613647, 0.6962425112724304, 0.7719510197639465, 0.811423122882843, 0.

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\jhluo\AppData\Local\Temp\ipykernel_9044\1938414908.py:2 in <cell line: 2>               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\jhluo\\AppData\\Local\\Temp\\ipykernel_9044\\1938414908.py'                          │
│                                                                                                  │
│ C:\Users\jhluo\AppData\Local\Temp\ipykernel_9044\1938414908.py:2 in <listcomp>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\jhluo\\AppData\\Local\\Temp\\ipykernel_9044\\1938414908.py'                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'float' object is not iterable

### Extract reslut to csv file

In [17]:
print(test_df.columns)

Index(['topic', 'premise', 'hypothesis', 'label'], dtype='object')


In [19]:
add_df = pd.DataFrame({
    'prediction': all_predictions,
    'probability':all_probabilities
})
result_df = pd.concat([test_df,add_df],axis=1)
print(result_df)

                                                  topic  \
0        Routine child vaccinations should be mandatory   
1        Routine child vaccinations should be mandatory   
2        Routine child vaccinations should be mandatory   
3        Routine child vaccinations should be mandatory   
4        Routine child vaccinations should be mandatory   
...                                                 ...   
3421  Social media platforms should be regulated by ...   
3422     Routine child vaccinations should be mandatory   
3423     Routine child vaccinations should be mandatory   
3424     Routine child vaccinations should be mandatory   
3425     Routine child vaccinations should be mandatory   

                                                premise  \
0      Routine childhood vaccinations  should be man...   
1      Routine childhood vaccinations  should be man...   
2      Routine childhood vaccinations  should be man...   
3      Routine childhood vaccinations  should be man...

In [20]:
result_df.to_csv('./result/KPM_official_test.csv', index=False)
print(f"DataFrame has been saved to {'./result/KPM_official_test.csv'}")

DataFrame has been saved to ./result/KPM_official_test.csv


In [13]:
# print(all_probabilities)
new_test_df = pd.read_csv('./result/KPM_official_test.csv')
print(new_test_df)

                                                  topic  \
0        Routine child vaccinations should be mandatory   
1        Routine child vaccinations should be mandatory   
2        Routine child vaccinations should be mandatory   
3        Routine child vaccinations should be mandatory   
4        Routine child vaccinations should be mandatory   
...                                                 ...   
3421  Social media platforms should be regulated by ...   
3422     Routine child vaccinations should be mandatory   
3423     Routine child vaccinations should be mandatory   
3424     Routine child vaccinations should be mandatory   
3425     Routine child vaccinations should be mandatory   

                                                premise  \
0      Routine childhood vaccinations  should be man...   
1      Routine childhood vaccinations  should be man...   
2      Routine childhood vaccinations  should be man...   
3      Routine childhood vaccinations  should be man...

In [16]:
new_test_df['logit'] = all_probabilities
print(new_test_df)
new_test_df.to_csv('./result/KPM_official_test.csv', index=False)
print(f"DataFrame has been saved to {'./result/KPM_official_test.csv'}")

                                                  topic  \
0        Routine child vaccinations should be mandatory   
1        Routine child vaccinations should be mandatory   
2        Routine child vaccinations should be mandatory   
3        Routine child vaccinations should be mandatory   
4        Routine child vaccinations should be mandatory   
...                                                 ...   
3421  Social media platforms should be regulated by ...   
3422     Routine child vaccinations should be mandatory   
3423     Routine child vaccinations should be mandatory   
3424     Routine child vaccinations should be mandatory   
3425     Routine child vaccinations should be mandatory   

                                                premise  \
0      Routine childhood vaccinations  should be man...   
1      Routine childhood vaccinations  should be man...   
2      Routine childhood vaccinations  should be man...   
3      Routine childhood vaccinations  should be man...

### Make Test Dataset

In [30]:
argument_df = pd.read_csv('./data/arguments_test.csv')
print(argument_df.columns)

Index(['arg_id', 'argument', 'topic', 'stance'], dtype='object')


In [31]:
kp_df = pd.read_csv('./data/key_points_test.csv')
print(kp_df.columns)

Index(['key_point_id', 'key_point', 'topic', 'stance'], dtype='object')


In [37]:
test_data = []
for index,row in argument_df.iterrows():
    filtered_kp = kp_df[(kp_df['topic'] == row['topic']) & (kp_df['stance'] == row['stance'])]
    filtered_kp_list = filtered_kp.values.tolist()
    for line in filtered_kp_list:
        test_data.append([row['topic'],row['stance'],row['arg_id'],row['argument'],line[0],line[1]])
print(len(test_data))

header = ['topic','stance','arg_id','argument','key_point_id','key_point']
df = pd.DataFrame(test_data, columns=header)
print(df)
print(df.columns)



3923
                                               topic  stance     arg_id  \
0     Routine child vaccinations should be mandatory      -1    arg_0_0   
1     Routine child vaccinations should be mandatory      -1    arg_0_0   
2     Routine child vaccinations should be mandatory      -1    arg_0_0   
3     Routine child vaccinations should be mandatory      -1    arg_0_0   
4     Routine child vaccinations should be mandatory      -1    arg_0_1   
...                                              ...     ...        ...   
3918            The USA is a good country to live in       1  arg_2_209   
3919            The USA is a good country to live in       1  arg_2_209   
3920            The USA is a good country to live in       1  arg_2_209   
3921            The USA is a good country to live in       1  arg_2_209   
3922            The USA is a good country to live in       1  arg_2_209   

                                               argument key_point_id  \
0     Routine child va

In [36]:
df.to_csv('./data/KPM_test_data.csv', index=False)
print(f"DataFrame has been saved to {'./data/KPM_test_data.csv'}")

DataFrame has been saved to ./data/KPM_test_data.csv


### Prediction

In [8]:
#Add new header confidence_score
df = pd.read_csv('./data/KPM_test_data.csv')
print(df.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'topic', 'stance', 'arg_id', 'argument',
       'key_point_id', 'key_point', 'confidence_score'],
      dtype='object')


In [9]:
# df['confidence_score'] = ''
# df.to_csv('./data/KPM_test_data.csv')
# print(f"DataFrame has been saved to {'./data/KPM_test_data.csv'}")

In [11]:
# Assuming you have already trained the model and saved a checkpoint
checkpoint_path = './checkpoint/nli_model-epoch=01-val_loss=0.37.ckpt'
# Load the model from the checkpoint
model_name = "cross-encoder/nli-distilroberta-base"
num_classes = 2
max_length = 512
batch_size = 16
learning_rate = 5e-05
weight_decay = 0.001

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_classes, ignore_mismatched_sizes = True)
loaded_model = KPM.load_from_checkpoint(model=model, checkpoint_path=checkpoint_path)
print("hello world")

# Set the model to evaluation mode
loaded_model.eval()


# Make predictions on the test set with probabilities

all_probabilities = []
df = pd.read_csv('./data/KPM_test_data.csv')
for index, row in df.iloc[:2].iterrows():
    premise = row['argument']+tokenizer.sep_token+row['topic']
    hypothesis = row['key_point']+tokenizer.sep_token+row['topic']
    encoding = tokenizer(
            premise,
            hypothesis,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
    with torch.no_grad():
        logits = loaded_model(**encoding).logits
        print(F.softmax(logits, dim=1).tolist()[0])
        probabilities = F.softmax(logits, dim=1).tolist()[0][0]
    print(str(index)+": "+str(probabilities))
    df.at[index,'confidence_score'] = probabilities
    # all_probabilities.append(probabilities)

df.to_csv('./data/KPM_test_data.csv', index=False)
print("Add new data successfully!!!!!")
# print("Finished")
# print(all_probabilities)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


hello world
[0.04978947713971138, 0.9502105712890625]
0: 0.04978947713971138
[0.9856218695640564, 0.01437812577933073]
1: 0.9856218695640564


### Make output json file for automatically evaluating

In [10]:
argument_df = pd.read_csv('./data/arguments_test.csv')
print(len(argument_df))
print(argument_df.columns)

723
Index(['arg_id', 'argument', 'topic', 'stance'], dtype='object')


In [19]:
test_df = pd.read_csv('./data/KPM_test_data.csv')
# test_df['confidence_score'] = 1- test_df['confidence_score']
print(len(test_df))
print(test_df)

3923
      Unnamed: 0                                           topic  stance  \
0              0  Routine child vaccinations should be mandatory      -1   
1              1  Routine child vaccinations should be mandatory      -1   
2              2  Routine child vaccinations should be mandatory      -1   
3              3  Routine child vaccinations should be mandatory      -1   
4              4  Routine child vaccinations should be mandatory      -1   
...          ...                                             ...     ...   
3918        3918            The USA is a good country to live in       1   
3919        3919            The USA is a good country to live in       1   
3920        3920            The USA is a good country to live in       1   
3921        3921            The USA is a good country to live in       1   
3922        3922            The USA is a good country to live in       1   

         arg_id                                           argument  \
0       arg_

In [20]:
test_df = test_df.drop(test_df.columns[0], axis=1)
print(test_df)

                                               topic  stance     arg_id  \
0     Routine child vaccinations should be mandatory      -1    arg_0_0   
1     Routine child vaccinations should be mandatory      -1    arg_0_0   
2     Routine child vaccinations should be mandatory      -1    arg_0_0   
3     Routine child vaccinations should be mandatory      -1    arg_0_0   
4     Routine child vaccinations should be mandatory      -1    arg_0_1   
...                                              ...     ...        ...   
3918            The USA is a good country to live in       1  arg_2_209   
3919            The USA is a good country to live in       1  arg_2_209   
3920            The USA is a good country to live in       1  arg_2_209   
3921            The USA is a good country to live in       1  arg_2_209   
3922            The USA is a good country to live in       1  arg_2_209   

                                               argument key_point_id  \
0     Routine child vaccina

In [23]:
# test_df.to_csv('./data/KPM_test_data.csv', index=False)
# print("Add new data successfully!!!!!")

Add new data successfully!!!!!


In [11]:
import pandas as pd
test_df = pd.read_csv('./data/KPM_test_data.csv')
print(test_df.columns)

Index(['topic', 'stance', 'arg_id', 'argument', 'key_point_id', 'key_point',
       '4_7_8_15', '0_1_2_3', '5_6_9_10', '11_12_13_14', '16_17_18_19',
       '20_21_22_23', '24_25_26_27', 'avg', 'gpt4', 'llama2', 'qwen'],
      dtype='object')


In [12]:
output_data = []
arg_id = argument_df['arg_id'].values.tolist()
for arg in arg_id:
    filtered_df = test_df[test_df['arg_id'] == arg]
    kp_id = filtered_df['key_point_id'].values.tolist()
    score = filtered_df['llama2'].values.tolist()
    result_dict = dict(zip(kp_id, score))
    output_data.append(result_dict)
final_result = dict(zip(arg_id,output_data))
print(final_result)

{'arg_0_0': {'kp_0_0': 0.2, 'kp_0_1': 0.2, 'kp_0_2': 0.5, 'kp_0_3': 0.2}, 'arg_0_1': {'kp_0_0': 0.7, 'kp_0_1': 0.6, 'kp_0_2': 0.5, 'kp_0_3': 0.6}, 'arg_0_2': {'kp_0_0': 0.2, 'kp_0_1': 0.2, 'kp_0_2': 0.2, 'kp_0_3': 0.2}, 'arg_0_3': {'kp_0_0': 0.5, 'kp_0_1': 0.4, 'kp_0_2': 0.5, 'kp_0_3': 0.5}, 'arg_0_4': {'kp_0_0': 0.7, 'kp_0_1': 0.7, 'kp_0_2': 0.7, 'kp_0_3': 0.7}, 'arg_0_5': {'kp_0_0': 0.2, 'kp_0_1': 0.2, 'kp_0_2': 0.2, 'kp_0_3': 0.2}, 'arg_0_6': {'kp_0_0': 0.2, 'kp_0_1': 0.2, 'kp_0_2': 0.2, 'kp_0_3': 0.2}, 'arg_0_7': {'kp_0_0': 0.3, 'kp_0_1': 0.3, 'kp_0_2': 0.3, 'kp_0_3': 0.2}, 'arg_0_8': {'kp_0_0': 0.2, 'kp_0_1': 0.3, 'kp_0_2': 0.5, 'kp_0_3': 0.2}, 'arg_0_9': {'kp_0_0': 0.2, 'kp_0_1': 0.2, 'kp_0_2': 0.3, 'kp_0_3': 0.2}, 'arg_0_10': {'kp_0_0': 0.2, 'kp_0_1': 0.2, 'kp_0_2': 0.2, 'kp_0_3': 0.2}, 'arg_0_11': {'kp_0_0': 0.7, 'kp_0_1': 0.2, 'kp_0_2': 0.2, 'kp_0_3': 0.2}, 'arg_0_12': {'kp_0_0': 0.2, 'kp_0_1': 0.2, 'kp_0_2': 0.5, 'kp_0_3': 0.2}, 'arg_0_13': {'kp_0_0': 0.7, 'kp_0_1': 0.3, 'kp_

In [13]:
import json
file_path = './result/llama2.json'
with open(file_path, 'w') as json_file:
    json.dump(final_result, json_file)
print(f"JSON file '{file_path}' created successfully.")

JSON file './result/llama2.json' created successfully.
